In [166]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.figure_factory as ff

from sklearn.metrics import r2_score
from tqdm import tqdm
from glob import glob
from plotly.offline import iplot
from collections import Counter
from IPython.core.interactiveshell import InteractiveShell
from sklearn.linear_model import LinearRegression
from statsmodels.stats.weightstats import ttest_ind, ztest


InteractiveShell.ast_node_interactivity = 'all'
pd.options.display.max_columns = 999
%matplotlib inline

In [2]:
paths = glob('./data/*.csv')
paths

['./data/data.csv', './data/preprocess.csv']

In [3]:
df = pd.read_csv(paths[1])
df.head()
df.describe()
df.info()

,วันที่,เวลา,เพศ,ช่วงอายุ,จำนวนชิ้นงาน,จำนวนหน้าต่อชิ้นงาน,จำนวนหน้ารวม,ต่อชิ้นงาน,ต่อบุคคล,ต่อจำนวนหน้า,ระยะเวลาจัดเตรียม(วินาที)
0,22/04/2019,8:18,ญ,แม่บ้าน,1,25,25,10.0,10,0.400000,420.0
1,22/04/2019,8:27,ญ,นักศึกษา,1,208,208,114.0,114,0.548077,1802.0
2,22/04/2019,8:37,ช,นักศึกษา,1,300,300,150.0,150,0.500000,30.0
3,22/04/2019,12:10,ญ,นักศึกษา,2,300,600,150.0,300,0.500000,30.0
4,22/04/2019,12:19,ญ,นักศึกษา,1,4,4,6.0,6,1.500000,66.0


,จำนวนชิ้นงาน,จำนวนหน้าต่อชิ้นงาน,จำนวนหน้ารวม,ต่อชิ้นงาน,ต่อบุคคล,ต่อจำนวนหน้า,ระยะเวลาจัดเตรียม(วินาที)
count,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000
mean,4.157895,38.394737,83.789474,19.378947,41.184211,0.600133,127.973684
std,12.775666,84.601623,219.649864,42.781703,109.191093,0.265835,293.304241
min,1.000000,1.000000,1.000000,1.000000,1.000000,0.333333,10.000000
25%,1.000000,3.000000,6.000000,2.000000,4.000000,0.400000,25.000000
50%,1.000000,7.000000,16.500000,3.000000,9.500000,0.500000,39.000000
75%,2.000000,25.000000,29.500000,10.000000,12.000000,0.666667,121.000000
max,80.000000,300.000000,1200.000000,150.000000,600.000000,1.500000,1802.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 11 columns):
วันที่                       38 non-null object
เวลา                         38 non-null object
เพศ                          38 non-null object
ช่วงอายุ                     38 non-null object
จำนวนชิ้นงาน                 38 non-null int64
จำนวนหน้าต่อชิ้นงาน          38 non-null int64
จำนวนหน้ารวม                 38 non-null int64
ต่อชิ้นงาน                   38 non-null float64
ต่อบุคคล                     38 non-null int64
ต่อจำนวนหน้า                 38 non-null float64
ระยะเวลาจัดเตรียม(วินาที)    38 non-null float64
dtypes: float64(3), int64(4), object(4)
memory usage: 3.3+ KB


# Summary

วันละ 9 ชม. <br>
รายได้กรณีทำงานตลอดทั้งวัน <br>
วันที่ 1: 7752 <br>
วันที่ 2: 282 <br>
วันที่ 3: 1356 <br>
รายได้เฉลี่ยต่อวัน: 3130 <br>

จำนวนกระดาษที่ใช้กรณีทำงานตลอดวัน <br>
วันที่ 1: 15462 <br>
วันที่ 2: 612 <br>
วันที่ 3: 3030 <br>
กระดาษเฉลี่ยต่อวัน: 6368 <br>

ค่าบริการเฉลี่ยต่อหน้า: 0.6001328872 <br>

# Time variable

In [113]:
Counter(df['วันที่'])

Counter({'22/04/2019': 13, '24/04/2019': 8, '26/04/2019': 17})

In [18]:
df.groupby(['วันที่'])[['จำนวนชิ้นงาน', 'จำนวนหน้าต่อชิ้นงาน', 'จำนวนหน้ารวม', 'ระยะเวลาจัดเตรียม(วินาที)']].sum()

,จำนวนชิ้นงาน,จำนวนหน้าต่อชิ้นงาน,จำนวนหน้ารวม,ระยะเวลาจัดเตรียม(วินาที)
วันที่,,,,
22/04/2019,22,1297,2577,3623.0
24/04/2019,21,35,102,562.0
26/04/2019,115,127,505,678.0


In [131]:
df_1 = df.loc[df['วันที่'] == '22/04/2019']
df_2 = df.loc[df['วันที่'] == '24/04/2019']
df_3 = df.loc[df['วันที่'] == '26/04/2019']

data = [
    np.log(df_1['จำนวนหน้ารวม']),
    np.log(df_2['จำนวนหน้ารวม']),
    np.log(df_3['จำนวนหน้ารวม'])
]
date = [
    '22/04/2019',
    '24/04/2019',
    '26/04/2019'
]

colors = ['#393E46', '#2BCDC1', '#F66095']

fig = ff.create_distplot(data ,date, bin_size=1, curve_type='normal', colors=colors, show_rug=False)
fig['layout'].update(title='No. Pages Distribution')
iplot(fig)

In [128]:
df_1 = df.loc[df['วันที่'] == '22/04/2019']
df_2 = df.loc[df['วันที่'] == '24/04/2019']
df_3 = df.loc[df['วันที่'] == '26/04/2019']

data = [
    np.log(df_1['ต่อจำนวนหน้า']),
    np.log(df_2['ต่อจำนวนหน้า']),
    np.log(df_3['ต่อจำนวนหน้า'])
]
date = [
    '22/04/2019',
    '24/04/2019',
    '26/04/2019'
]

colors = ['#393E46', '#2BCDC1', '#F66095']

fig = ff.create_distplot(data ,date, bin_size=.5, curve_type='normal', colors=colors, show_rug=False)
fig['layout'].update(title='Price per page Distribution')
iplot(fig)

In [145]:
data = [
    go.Box(y=df.loc[df['วันที่'] == '22/04/2019', 'ต่อจำนวนหน้า'], name='22/04/2019', marker=dict(color=colors[0])),
    go.Box(y=df.loc[df['วันที่'] == '24/04/2019', 'ต่อจำนวนหน้า'], name='24/04/2019', marker=dict(color=colors[1])),
    go.Box(y=df.loc[df['วันที่'] == '26/04/2019', 'ต่อจำนวนหน้า'], name='26/04/2019', marker=dict(color=colors[2]))
]
layout = go.Layout(title='Price per page distribution')

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [146]:
data = [
    go.Box(y=df.loc[df['วันที่'] == '22/04/2019', 'จำนวนหน้ารวม'], name='22/04/2019', marker=dict(color=colors[0])),
    go.Box(y=df.loc[df['วันที่'] == '24/04/2019', 'จำนวนหน้ารวม'], name='24/04/2019', marker=dict(color=colors[1])),
    go.Box(y=df.loc[df['วันที่'] == '26/04/2019', 'จำนวนหน้ารวม'], name='26/04/2019', marker=dict(color=colors[2]))
]
layout = go.Layout(title='No. Pages distribution')

fig = go.Figure(data=data, layout=layout)

iplot(fig)

ถ้าไม่คำนึงถึง outlier จะเห็นว่าวันที่ 22 โดยภาพรวมมีจำนวนแผ่นที่ไปถ่ายเอกสารมากกว่าวันอื่นๆ และราคาโดยภาพรวมต่อแผ่นถูกกว่าวันอื่นๆ

# Sex variable

In [147]:
Counter(df['เพศ'])

Counter({'ญ': 16, 'ช': 22})

In [148]:
df.groupby(['เพศ'])[['จำนวนชิ้นงาน', 'จำนวนหน้าต่อชิ้นงาน', 'จำนวนหน้ารวม', 'ระยะเวลาจัดเตรียม(วินาที)']].sum()

,จำนวนชิ้นงาน,จำนวนหน้าต่อชิ้นงาน,จำนวนหน้ารวม,ระยะเวลาจัดเตรียม(วินาที)
เพศ,,,,
ช,116,851,2176,1821.0
ญ,42,608,1008,3042.0


In [149]:
df_male = df.loc[df['เพศ'] == 'ช']
df_female = df.loc[df['เพศ'] == 'ญ']

data = [
    np.log(df_male['จำนวนหน้ารวม']),
    np.log(df_female['จำนวนหน้ารวม'])
]
colors = ['#7FA6EE', '#F64E8B']

fig = ff.create_distplot(data ,['Male', 'Female'], bin_size=1.25, curve_type='normal', colors=colors, show_rug=False)
fig['layout'].update(title='No. Pages Distribution')
iplot(fig)

In [150]:
df_male = df.loc[df['เพศ'] == 'ช']
df_female = df.loc[df['เพศ'] == 'ญ']

data = [
    np.log(df_male['ต่อจำนวนหน้า']),
    np.log(df_female['ต่อจำนวนหน้า'])
]
colors = ['#7FA6EE', '#F64E8B']

fig = ff.create_distplot(data ,['Male', 'Female'], bin_size=.4, curve_type='normal', colors=colors, show_rug=False)
fig['layout'].update(title='Price per page Distribution')
iplot(fig)

In [158]:
data = [
    go.Box(y=df.loc[(df['เพศ'] == 'ช'), 'ต่อจำนวนหน้า'], name='ชาย', marker=dict(color=colors[0])),
    go.Box(y=df.loc[(df['เพศ'] == 'ญ'), 'ต่อจำนวนหน้า'], name='หญิง', marker=dict(color=colors[1])),
]
layout = go.Layout(title='Price per page distribution')

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [159]:
data = [
    go.Box(y=df.loc[(df['เพศ'] == 'ช') & (df['จำนวนหน้ารวม'] < 200), 'จำนวนหน้ารวม'], name='ชาย', marker=dict(color=colors[0])),
    go.Box(y=df.loc[(df['เพศ'] == 'ญ') & (df['จำนวนหน้ารวม'] < 200), 'จำนวนหน้ารวม'], name='หญิง', marker=dict(color=colors[1])),
]
layout = go.Layout(title='No. Pages distribution')

fig = go.Figure(data=data, layout=layout)

iplot(fig)

เมื่อมองโดยภาพรวมแล้วเพศชาย โดยเฉลี่ยจะถ่ายเอกสารตกราคาต่อแผ่นสูงกว่า หญิงนิดหน่อย 555 และจำนวนแผ่นที่ถ่ายมองโดยภาพรวมแล้วพอๆกัน

# Hypothesis Testing

ตั้งสมมุติฐานว่า ราคาต่อแผ่นโดยเฉลี่ยของเพศชาย มากกว่าเพศหญิงหรือไม่ที่ระดับนัยสำคัญ 0.05

$$
H_{0}: µ_{M} \le µ_{F}
$$
$$
H_{A}: µ_{M} \gt µ_{F}
$$

In [164]:
df_male   = df.loc[(df['เพศ'] == 'ช'), 'ต่อจำนวนหน้า']
df_female = df.loc[(df['เพศ'] == 'ญ'), 'ต่อจำนวนหน้า']

In [169]:
ttest_ind(df_male, df_female, alternative='larger')[1]

0.45958511113853256

P-value > 0.05: Fail to reject $H_{0}$

∴ ยอมรับ $H_{0}$ ราคาต่อแผ่นโดยเฉลี่ยของเพศชาย ไม่ได้มากกว่าเพศหญิงที่ีระดับนัยสำคัญ 0.05

# Regression

In [26]:
data = [
    go.Scatter(
        x = df['จำนวนหน้ารวม'],
        y = df['ต่อจำนวนหน้า'],
        mode = 'markers'
    )
]
layout = go.Layout(
    title='Scatter plot between No.Pages & Price per page',
    xaxis=dict(
        title='No. Pages'
    ),
    yaxis=dict(
        title='Price per page'
    ),
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [34]:
data = [
    go.Box(y=df['จำนวนหน้ารวม'], name='No. Pages'),
]
layout = go.Layout(title='No. Pages')

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [33]:
data = [
    go.Box(y=df['ต่อจำนวนหน้า'], name='Price per page'),
]
layout = go.Layout(title='Price per page')

fig = go.Figure(data=data, layout=layout)

iplot(fig)

### Remove outlier with IQR

In [160]:
tmp = df[['ต่อจำนวนหน้า', 'จำนวนหน้ารวม']]
tmp.describe()

,ต่อจำนวนหน้า,จำนวนหน้ารวม
count,38.000000,38.000000
mean,0.600133,83.789474
std,0.265835,219.649864
min,0.333333,1.000000
25%,0.400000,6.000000
50%,0.500000,16.500000
75%,0.666667,29.500000
max,1.500000,1200.000000



| ''          | ต่อจำนวนหน้า  | จำนวนหน้ารวม |
|-------------|-------------|-------------|
|IQR          |  0.266      |  23.5       |
|1.5 IQR      |  0.4        |  35.25      |
|lower bound  |  0          |   -         |
|upper bound  |  1          |   64.75     |

In [161]:
tmp = tmp.loc[( (tmp['ต่อจำนวนหน้า'] >= 0 ) & (tmp['ต่อจำนวนหน้า'] <= 1) ) & ( (tmp['จำนวนหน้ารวม'] >= 0 ) & (tmp['จำนวนหน้ารวม'] <= 64.75) )]

In [162]:
data = [
    go.Scatter(
        x = tmp['จำนวนหน้ารวม'],
        y = tmp['ต่อจำนวนหน้า'],
        mode = 'markers'
    )
]
layout = go.Layout(
    title='Scatter plot between No.Pages & Price per page',
    xaxis=dict(
        title='No. Pages'
    ),
    yaxis=dict(
        title='Price per page'
    ),
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

### Modeling

In [91]:
X = tmp['จำนวนหน้ารวม'].values.reshape(-1, 1)
y = tmp['ต่อจำนวนหน้า'].values.reshape(-1, 1)

In [92]:
model = LinearRegression().fit(X, y)

In [93]:
pred = model.predict(X).ravel()

In [172]:
data = [
    go.Scatter(
        x = tmp['จำนวนหน้ารวม'],
        y = tmp['ต่อจำนวนหน้า'],
        mode = 'markers',
        name='data'
    ),
    go.Scatter(
        x = tmp['จำนวนหน้ารวม'],
        y = pred,
        name='regression line'
    )
]
layout = go.Layout(
    title='Regression',
    xaxis=dict(
        title='No. Pages'
    ),
    yaxis=dict(
        title='Price per page'
    ),
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

### Equation of Regression



In [171]:
model.coef_
model.intercept_

array([[-0.00871453]])

array([0.73960605])


$$
Y^{\hat{}} = -0.0087X + 0.739
$$


In [89]:
r2_score(y, pred)

0.2938668126863828

$$
R^{2} = 0.29
$$

∴ เราสามารถใช้สมการในการอธิบายความสัมพันธ์ของตัวแปรได้ด้วยความเชื่อมั่นอยู่ที่ 29%